In [1]:
import cv2
from mtcnn.mtcnn import MTCNN
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.5.1 (SDL 2.28.2, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5")

# Without Attention

In [3]:
mixer.init()
sound = mixer.Sound(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\alarm.wav")

cap = cv2.VideoCapture(0)
Score = 0
while True:
    ret, frame = cap.read()
#     print(frame.shape)
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    
    cv2.rectangle(frame, (0, height-50), (200,height), (0,0,0), thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, pt1=(x,y), pt2=(x+w, y+h), color=(255,0,0), thickness=3)
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame, pt1=(ex,ey), pt2=(ex+ew, ey+eh), color=(255,0,0), thickness=3)
        #preprocessing steps
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye, axis = 0)
        
        #Model prediction
        prediction = model.predict(eye)
        
        #If eyes are closed
        if prediction[0][0] > 0.50:
            cv2.putText(frame, 'Closed', (10, height-20), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1, color = (255,255,255), thickness = 1)
            
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1, color = (255,255,255), thickness = 1)
            
            Score = Score + 1
            if(Score > 5):
                try:
                    sound.play()
                except: pass
            
        
        #If eyes are open
        elif prediction[0][1] > 0.50:
            cv2.putText(frame, 'Open', (10, height-20), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1, color = (255,255,255), thickness = 1)
        
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1, color = (255,255,255), thickness = 1)
            
            Score = Score - 1
            if(Score < 0):
                Score=0
            
    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 34ms/step


# With Attention

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from pygame import mixer
import time

# Load your drowsiness detection model
model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5")

mixer.init()
sound = mixer.Sound(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\alarm.wav")

cap = cv2.VideoCapture(0)
Score = 0
Attention = 0  # Initialize the attention score
frame_count = 0  # Initialize a frame counter
start_time = time.time()  # Initialize start time

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    if len(eyes)== 0:
        # Increment attention score when no faces or eyes are detected, but only every second
        current_time = time.time()
        if current_time - start_time >= 1:
            Attention += 1
            start_time = current_time
            if(Attention >= 5):
                try:
                    sound.play()
                except:
                    pass

    else:
        Attention = 0  # Reset attention score when faces or eyes are detected

    # Display attention score
    cv2.putText(frame, 'Attention: ' + str(Attention), (10, height - 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 2)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=3)

    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(frame, pt1=(ex, ey), pt2=(ex + ew, ey + eh), color=(255, 0, 0), thickness=3)
        # preprocessing steps
        eye = frame[ey:ey + eh, ex:ex + ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)

        # Model prediction
        prediction = model.predict(eye)

        # If eyes are closed
        if prediction[0][0] > 0.50:
#             cv2.putText(frame, 'Closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)

            cv2.putText(frame, 'Score' + str(Score), (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)

            Score += 1
            if Score >= 4:
                try:
                    sound.play()
                except:
                    pass

        # If eyes are open
        elif prediction[0][1] > 0.50:
#             cv2.putText(frame, 'Open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)

            cv2.putText(frame, 'Score' + str(Score), (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)

            Score -= 1
            if Score < 0:
                Score = 0

    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 35ms/step
